In [1]:
DATA_DIR = 'data/'
RESULT_DIR = 'results/original_vs_updated/'
import os
# Create the directories
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

In [2]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("dl2-2024/dl-2024")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv(DATA_DIR + "project.csv")

In [41]:
import pandas as pd
from tabulate import tabulate
import numpy as np

# Read the CSV file
runs_df = pd.read_csv(DATA_DIR + "project.csv")

# Extract and flatten the summary and config columns
summary_df = pd.json_normalize(runs_df['summary'].apply(eval))
config_df = pd.json_normalize(runs_df['config'].apply(eval))

# Combine the flattened summary and config data
results_df = pd.concat([runs_df[['name']], config_df, summary_df], axis=1)

# Filter results based on `softmax` and `use_underscore` settings
original_code = results_df[(results_df['softmax'] == True) & (results_df['use_underscore'] == True)]
updated_code = results_df[(results_df['softmax'] == False) & (results_df['use_underscore'] == False)]

# Function to get the specified metric for each dataset, model, and transformation combination
def get_metric(df, dataset_name, model_name, transformation, metric):
    result = df[(df['dataset_name'] == dataset_name) & 
                (df['model_name'] == model_name) & 
                (df['data_transformations'] == transformation)][metric]
    return np.round(result.iloc[0], 2) if not result.empty else None

# Create a summary DataFrame for each combination
architectures = ['RN50']  # Add more architectures if needed
transformations = ['flip', 'rot90']
datasets = ['CIFAR100', 'ISIC2018']

# Initialize the summary data structure
summary_data = {
    "Architecture-Transformation": []
}

# Initialize columns for each dataset and metric
metrics = ["val_top1_acc", "final_top1_acc"]
for dataset in datasets:
    for metric in metrics:
        summary_data[f"{dataset} Original {metric.replace('_', ' ').title()}"] = []
        summary_data[f"{dataset} Updated {metric.replace('_', ' ').title()}"] = []

# Populate the summary data
for architecture in architectures:
    for transformation in transformations:
        combination = f"{architecture} {transformation}"
        summary_data["Architecture-Transformation"].append(combination)
        for dataset in datasets:
            for metric in metrics:
                summary_data[f"{dataset} Original {metric.replace('_', ' ').title()}"].append(
                    get_metric(original_code, dataset, architecture, transformation, metric))
                summary_data[f"{dataset} Updated {metric.replace('_', ' ').title()}"].append(
                    get_metric(updated_code, dataset, architecture, transformation, metric))

summary_df = pd.DataFrame(summary_data)

# Save the summary table to a new CSV file
# summary_df.to_csv(RESULT_DIR + "final_accuracy_summary.csv", index=False)

# Convert the dataframe to a markdown table
markdown_table = tabulate(summary_df, headers='keys', tablefmt='pipe')

# Save the markdown table to a file
with open(RESULT_DIR + "original_vs_updated_summary.md", "w") as file:
    file.write(markdown_table)

# Display the markdown table
print(markdown_table)


|    | Architecture-Transformation   |   CIFAR100 Original Prefinetune Top1 Acc |   CIFAR100 Updated Prefinetune Top1 Acc |   CIFAR100 Original Finetune Top1 Acc |   CIFAR100 Updated Finetune Top1 Acc |   ISIC2018 Original Prefinetune Top1 Acc |   ISIC2018 Updated Prefinetune Top1 Acc |   ISIC2018 Original Finetune Top1 Acc |   ISIC2018 Updated Finetune Top1 Acc |
|---:|:------------------------------|-----------------------------------------:|----------------------------------------:|--------------------------------------:|-------------------------------------:|-----------------------------------------:|----------------------------------------:|--------------------------------------:|-------------------------------------:|
|  0 | RN50 flip                     |                                    36.12 |                                   37.94 |                                 52.04 |                                56.49 |                                    12.94 |                     